In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from skimage.measure import EllipseModel
import pandas as pd

# ---- Example placeholder data ----
# Replace this with actual intersection points from your mesh
# Example: a roughly elliptical point cloud
np.random.seed(0)
t = np.linspace(0, 2 * np.pi, 100)
x = 10 * np.cos(t) + np.random.normal(0, 0.5, 100)
y = 5 * np.sin(t) + np.random.normal(0, 0.5, 100)
points_2d = np.vstack((x, y)).T

# ---- Method 1: PCA ----
def compute_pca_diameters(points_2d):
    centered = points_2d - np.mean(points_2d, axis=0)
    pca = PCA(n_components=2)
    pca.fit(centered)
    major_length = 2 * np.sqrt(pca.explained_variance_[0])
    minor_length = 2 * np.sqrt(pca.explained_variance_[1])
    major_dir = pca.components_[0] * major_length / 2
    minor_dir = pca.components_[1] * minor_length / 2
    origin = np.mean(points_2d, axis=0)

    return {
        "major_length": major_length,
        "minor_length": minor_length,
        "origin": origin,
        "major_dir": major_dir,
        "minor_dir": minor_dir
    }

# ---- Method 2: Ellipse Fitting ----
def compute_ellipse_diameters(points_2d):
    ellipse = EllipseModel()
    success = ellipse.estimate(points_2d)
    if success:
        xc, yc, a, b, theta = ellipse.params
        major_length = 2 * max(a, b)
        minor_length = 2 * min(a, b)
        return {
            "major_length": major_length,
            "minor_length": minor_length,
            "center": (xc, yc),
            "axes": (a, b),
            "theta": theta
        }
    else:
        return None

# ---- Run both methods ----
pca_result = compute_pca_diameters(points_2d)
ellipse_result = compute_ellipse_diameters(points_2d)

# ---- Plotting PCA ----
origin = pca_result["origin"]
maj = pca_result["major_dir"]
minr = pca_result["minor_dir"]

plt.figure(figsize=(6, 6))
plt.scatter(points_2d[:, 0], points_2d[:, 1], alpha=0.6, label="Intersection Points")
plt.plot([origin[0] - maj[0], origin[0] + maj[0]],
         [origin[1] - maj[1], origin[1] + maj[1]],
         color="blue", label="PCA Major Axis")
plt.plot([origin[0] - minr[0], origin[0] + minr[0]],
         [origin[1] - minr[1], origin[1] + minr[1]],
         color="red", label="PCA Minor Axis")

plt.text(origin[0] + maj[0], origin[1] + maj[1],
         f"Major: {pca_result['major_length']:.2f} mm", color='blue')
plt.text(origin[0] + minr[0], origin[1] + minr[1],
         f"Minor: {pca_result['minor_length']:.2f} mm", color='red')

plt.title("PCA-Based Diameter Estimation")
plt.axis("equal")
plt.legend()
plt.show()

# ---- Plotting Ellipse ----
if ellipse_result:
    xc, yc = ellipse_result["center"]
    a, b = ellipse_result["axes"]
    theta = ellipse_result["theta"]

    t = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = xc + a * np.cos(t) * np.cos(theta) - b * np.sin(t) * np.sin(theta)
    ellipse_y = yc + a * np.cos(t) * np.sin(theta) + b * np.sin(t) * np.cos(theta)

    plt.figure(figsize=(6, 6))
    plt.scatter(points_2d[:, 0], points_2d[:, 1], alpha=0.6, label="Intersection Points")
    plt.plot(ellipse_x, ellipse_y, 'g--', label="Fitted Ellipse")
    plt.text(xc + a, yc, f"Major: {ellipse_result['major_length']:.2f} mm", color='blue')
    plt.text(xc, yc + b, f"Minor: {ellipse_result['minor_length']:.2f} mm", color='red')
    plt.title("Ellipse Fit Diameter Estimation")
    plt.axis("equal")
    plt.legend()
    plt.show()
else:
    print("Ellipse fitting failed.")

# ---- Compare Results ----
results_df = pd.DataFrame({
    "Method": ["PCA", "Ellipse Fitting"],
    "Major Diameter (mm)": [pca_result["major_length"], ellipse_result["major_length"]],
    "Minor Diameter (mm)": [pca_result["minor_length"], ellipse_result["minor_length"]]
})

print(results_df)
